In [0]:
import io
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
from googleapiclient.discovery import build
from google.colab import files

In [0]:
import os
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
from PIL import Image
from scipy import misc
from skimage import color
from urllib.request import urlretrieve

In [0]:
!rm -rf checkpoint Test Train
!ls

In [0]:
!ls

In [0]:
!mkdir data

In [0]:
data_dir = 'data'

In [0]:
# NOT NEEDED
### Convert img from RGB to YCbCr
def rgb2ycbcr(im):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = im.dot(xform.T)
    ycbcr[:,:,[1,2]] += 128
    return np.uint8(ycbcr)

### Convert img from YCbCr to RGB
def ycbcr2rgb(im):
    xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    rgb = im.astype(np.float)
    rgb[:,:,[1,2]] -= 128
    return np.uint8(rgb.dot(xform.T))

In [0]:
### Function to measure PSNR
def psnr(x, y, maximum=255.0):
    return 20 * np.log10(maximum / np.sqrt(np.mean((x - y) ** 2)))

In [0]:
train_zip_url = 'https://cv.snu.ac.kr/research/VDSR/train_data.zip'
test_zip_url = 'https://cv.snu.ac.kr/research/VDSR/test_data.zip'

In [0]:
!ls

In [0]:
!rm -rf log
!rm -rf model
!mkdir model
!mkdir log
!ls

In [0]:
!wget https://cv.snu.ac.kr/research/VDSR/train_data.zip

In [0]:
!wget https://cv.snu.ac.kr/research/VDSR/test_data.zip

In [0]:
!unzip test_data.zip
!unzip train_data.zip

In [0]:
!mv 291 data/
!mv 91 data/
!mv B100 data/
!mv Set5 data/
!mv Set14 data/
!mv Urban100 data/

In [0]:
!ls data

#### upload previous checkpoint as zip

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
!unzip model_VDSR_ckpt24.zip

In [0]:
!ls data

In [0]:
train_benchmark = '91'   # ['91/', '291/']
test_benchmark = 'Set5'  # ['Set5/', 'Set14/', 'B100/', 'Urban100/']
batch_size = 64
patch_size = 41
scaling_factors = (2, 3, 4)
num_epoch = 1
weight_decay = 0.0001
gradient_clipping = 0.9

In [0]:
train_dir = os.path.join(data_dir, train_benchmark)
test_dir = os.path.join(data_dir, test_benchmark)

images, targets = [], []
test_images, test_targets = [], []

In [0]:
for image_file in os.listdir(train_dir): # INPUT AND PREPROCESS IMAGES
    image = misc.imread(os.path.join(train_dir, image_file))
    if len(image.shape) == 3:
        image = color.rgb2ycbcr(image)[:,:,0].astype(np.uint8)
    width, height = image.shape
    width = width - width % 12
    height = height - height % 12
    n_horizontal_patches = width // patch_size
    n_vertical_patches = height // patch_size
    image = image[:width, :height]
    
    for scaling_factor in scaling_factors:
        downscaled = misc.imresize(image, 1./scaling_factor, 'bicubic', mode='L')
        rescaled = misc.imresize(downscaled, float(scaling_factor), 'bicubic', mode='L')
        hr_image = image.astype(np.float32) / 255
        lr_image = np.clip(rescaled.astype(np.float32) / 255, 0.0, 1.0)
        
        for horizontal_patch in range(n_horizontal_patches):
            for vertical_patch in range(n_vertical_patches):
                h_start = horizontal_patch * patch_size
                v_start = vertical_patch * patch_size
                hr_patch = hr_image[h_start:h_start+patch_size, v_start:v_start+patch_size]
                lr_patch = lr_image[h_start:h_start+patch_size, v_start:v_start+patch_size]
                
                '''
                for _ in range(4):
                    hr_patch = np.rot90(hr_patch)
                    lr_patch = np.rot90(lr_patch)
                    targets.append(np.expand_dims(hr_patch, axis=2))
                    images.append(np.expand_dims(lr_patch, axis=2))
                
                lr_patch = np.fliplr(lr_patch)
                hr_patch = np.fliplr(hr_patch)
                
                for _ in range(4):
                    hr_patch = np.rot90(hr_patch)
                    lr_patch = np.rot90(lr_patch)
                    targets.append(np.expand_dims(hr_patch, axis=2))
                    images.append(np.expand_dims(lr_patch, axis=2))
                '''
                
                targets.append(np.expand_dims(hr_patch, axis=2))
                images.append(np.expand_dims(lr_patch, axis=2))

In [0]:
images = np.array(images)
targets = np.array(targets)
print(images.shape)

In [0]:
# SHUFFLE IMAGES
indices = list(range(len(images)))
np.random.shuffle(indices)
images = images[indices]
targets = targets[indices]

In [0]:
length = len(images)
length -= length % batch_size
images = images[:length]
targets = targets[:length]
print('Number of training images after batch approximation: [{}]'.format(length))

In [0]:
test_images = []
test_targets = []
test_images_scaled = [[] for s in range(len(scaling_factors))]
test_targets_scaled = [[] for s in range(len(scaling_factors))]
scaling_diff = min(scaling_factors)

for image_file in os.listdir(test_dir):
    image = misc.imread(os.path.join(test_dir, image_file))
    width, height = image.shape[0], image.shape[1]
    width = width - width % 12
    height = height - height % 12
    image = image[:width, :height]
    
    if len(image.shape) == 3:
        ycbcr = color.rgb2ycbcr(image)
        y = ycbcr[:,:,0].astype(np.uint8)
    else:
        y = image
    
    for scaling_factor in scaling_factors:
        downscaled = misc.imresize(y, 1./scaling_factor, 'bicubic', mode='L')
        rescaled = misc.imresize(downscaled, float(scaling_factor), 'bicubic', mode='L')
        
        if len(image.shape) == 3:
            lr_image = ycbcr
            lr_image[:,:,0] = rescaled
            lr_image = color.ycbcr2rgb(lr_image)
            lr_image = (np.clip(lr_image, 0.0, 1.0) * 255).astype(np.uint8)
        else:
            lr_image = rescaled
        
        test_images.append(lr_image)
        test_targets.append(image)
        #test_images_scaled[scaling_factor-scaling_diff].append(lr_image)
        #test_targets_scaled[scaling_factor-scaling_diff].append(image)
'''
for imgs in test_images_scaled:
    for img in imgs:
        test_images.append(img)
for imgs in test_targets_scaled:
    for img in imgs:
        test_targets.append(imgs)
'''

In [0]:
t_length = len(test_images)
print('Number of test images: [{}]'.format(t_length))

test_images = np.array(test_images)
test_targets = np.array(test_targets)

In [0]:
test_images.shape

In [0]:
num_layers = 20
k_size = 3
num_filters = 64
learning_rate_decay = 0.1
g_learning_rate = 1e-4
learning_rate_decay_step = 20

In [0]:
input = tf.placeholder(tf.float32)
ground_truth = tf.placeholder(tf.float32)
learning_rate = tf.placeholder(tf.float32, shape=[])
global_step = tf.Variable(0, trainable=False, name='global_step')

In [0]:
weights, biases = [], []

In [0]:
output = input
for i in range(num_layers):
  if i == 0:
    in_shape = 1
  else:
    in_shape = num_filters
  
  if i == num_layers - 1:
    out_shape = 1
  else:
    out_shape = num_filters
  
  weight = tf.Variable(tf.random_normal([k_size, k_size, in_shape, out_shape],
                                       stddev=np.sqrt(2/(k_size**2*in_shape))))
  bias = tf.Variable(tf.zeros([out_shape]))
  
  weights.append(weight)
  biases.append(bias)
  
  output = tf.nn.bias_add(tf.nn.conv2d(output, weight, strides=[1,1,1,1], padding='SAME'), bias)
  if i < num_layers - 1:
    output = tf.nn.relu(output)

residual = output
output = tf.add(output, input)

In [0]:
base_loss = tf.reduce_sum(tf.nn.l2_loss(tf.subtract(output, ground_truth)))
weight_loss = weight_decay * tf.reduce_sum(tf.stack([tf.nn.l2_loss(weight) for weight in weights]))
loss = base_loss + weight_loss
validation_score = tf.placeholder(tf.float32)

In [0]:
tf.summary.scalar('base loss', base_loss)
tf.summary.scalar('weight loss', weight_loss)
tf.summary.scalar('total loss', loss)
tf.summary.scalar('learning rate', learning_rate)
tf.summary.scalar('validation score', validation_score)
tf.summary.image('input', input, max_outputs=15)
tf.summary.image('output', output, max_outputs=15)
tf.summary.image('ground truth', ground_truth, max_outputs=15)
tf.summary.image('residual', residual, max_outputs=15)

In [0]:
for i in range(len(weights)):
    tf.summary.histogram('weights/layer #%d' % (i + 1), weights[i])
    tf.summary.histogram('biases/layer #%d' % (i + 1), biases[i])

In [0]:
summary_step = tf.summary.merge_all()
saver = tf.train.Saver(max_to_keep=10)

In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate)
gradients = optimizer.compute_gradients(loss)
clip_value = gradient_clipping / learning_rate
#capped_gradients = [(tf.clip_by_value(grad, -clip_value, clip_value), var) for grad, var in gradients]
capped_gradients = list(map(lambda grad: grad if grad[0] is None else (tf.clip_by_value(grad[0], -clip_value, clip_value), grad[1]), gradients))
train_step = optimizer.apply_gradients(capped_gradients, global_step=global_step)

In [0]:
checkpoint_path = os.path.join(data_dir, 'model')
model_path = os.path.join(checkpoint_path, 'model.ckpt')
log_path = os.path.join(data_dir, 'log')

In [0]:
summary_writer = tf.summary.FileWriter(log_path)

In [0]:
for path in [checkpoint_path, log_path]:
    if not os.path.exists(path):
        os.mkdir(path)

In [0]:
def predict(sess, output, images, targets, border=0):
    predictions = []
    psnr_arr = []
    
    for i in range(len(images)):
        image = images[i]
        if len(image.shape) == 3:
            image_ycbcr = color.rgb2ycbcr(image)
            image_y = image_ycbcr[:,:,0]
        else:
            image_y = image.copy()
        
        image_y = image_y.astype(np.float) / 255
        reshaped_image_y = np.array([np.expand_dims(image_y, axis=2)])
        #print(reshaped_image_y)
        prediction = output.eval(feed_dict={input: reshaped_image_y}, session=sess)[0]
        if np.isnan(prediction[0,0,0]):
            print('problem.....')
        prediction *= 255
        
        if len(targets[i].shape) == 3:
            target_y = color.rgb2ycbcr(targets[i])[:,:,0]
        else:
            target_y = targets[i].copy()
        #psnr_arr.append(psnr(np.array(prediction[:,:,0]),np.array(target_y)))
        '''20 * np.log10(maximum / np.sqrt(np.mean((x - y) ** 2)))'''
        psnr_val = 20 * np.log10(255.0 / np.sqrt(np.mean((prediction.squeeze() - target_y) ** 2)))
        #print(prediction[:,:,0],target_y)
        #print('val {}: [{}]'.format(i, psnr_val))
        psnr_arr.append(psnr_val)
            
        if len(image.shape) == 3:
            prediction = color.ycbcr2rgb(np.concatenate((prediction, image_ycbcr[:,:,1:3]), axis=2)) * 255
        else:
            prediction = prediction[:,:,0]
        
        prediction = np.clip(prediction, 0, 255).astype(np.uint8)
        predictions.append(prediction)
    return predictions, psnr_arr

In [0]:
config = tf.ConfigProto(device_count = {'GPU': 0})
config.gpu_options.allow_growth = True

In [0]:
num_epoch = 25

In [0]:
!mkdir output_vdsr

In [0]:
### SAVE IMAGES

images_completed = 0
epochs_completed = 0
t_images_completed = 0

increment_global_step_op = tf.assign(global_step, global_step+1)
assign_global_zero_op = tf.assign(global_step, tf.constant(0))

start_time = time.time()
epoch_time = start_time

with tf.Session(config=config) as sess:
    checkpoint = tf.train.get_checkpoint_state(checkpoint_path)
    if checkpoint and checkpoint.model_checkpoint_path:
        print('Restoring model...')
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print(global_step.value())
        print('Restore complete...')
        #sess.run(assign_global_zero_op)
    else:
        print('New model...')
        sess.run(tf.global_variables_initializer())
        print('Initialization complete...')
    
    print('Training...')
    while tf.train.global_step(sess, global_step) * batch_size < length * num_epoch:
        predictions, psnr = predict(sess, output, test_images, targets=test_targets)
        pp = np.array(predictions)
        #print(pp[0])
        #plt.imshow(pp[0])
        for i in range(15):
            misc.imsave('output_vdsr/image_{}.png'.format(i), pp[i])
        break
        '''
        batch = tf.train.global_step(sess, global_step)
        epoch = batch * batch_size / length
        
        x = images[images_completed: images_completed+batch_size]
        y = targets[images_completed: images_completed+batch_size]
        
        images_completed += batch_size
        
        if images_completed >= length:
            images_completed = 0
            epochs_completed += 1

        current_learning_rate = g_learning_rate * learning_rate_decay ** (epoch // learning_rate_decay_step)
        feed_dict = {input: x, ground_truth: y, learning_rate: current_learning_rate}
        
        print('batch no.: [{}], images completed: [{}/{}]'.format(batch, images_completed+batch_size, length))
        
        if batch * batch_size % length == 0:
            print('Processing epoch #{}, Total epoch time: [{}]'.format(epoch + 1, time.time()-epoch_time))
            epoch_time = time.time()
            sess.run(increment_global_step_op)
            predictions, psnr = predict(sess, output, test_images, targets=test_targets)
            feed_dict[validation_score] = np.mean(psnr)
            print('Current PSNR: [{}], Average PSNR: [{}]'.format(psnr, feed_dict[validation_score]))
            _, summary = sess.run([train_step, summary_step], feed_dict=feed_dict)
            saver.save(sess, model_path)
            summary_writer.add_summary(summary, epoch)
        else:
            print('Training epoch #{}, Time: [{}]'.format(epoch + 1, time.time()-start_time))
            sess.run([train_step], feed_dict=feed_dict)
            #predictions, psnr = predict(sess, output, test_images, targets=test_targets)
            #feed_dict[validation_score] = np.mean(psnr)
            #print('Current PSNR: [{}], Average PSNR: [{}]'.format(psnr, feed_dict[validation_score]))
        if batch % 10 == 0:
            saver.save(sess, model_path)
    saver.save(sess, model_path)
    '''
#num_epoch += 1

In [0]:
images_completed = 0
epochs_completed = 0
t_images_completed = 0

increment_global_step_op = tf.assign(global_step, global_step+1)
assign_global_zero_op = tf.assign(global_step, tf.constant(0))

start_time = time.time()
epoch_time = start_time

with tf.Session(config=config) as sess:
    checkpoint = tf.train.get_checkpoint_state(checkpoint_path)
    if checkpoint and checkpoint.model_checkpoint_path:
        print('Restoring model...')
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print(global_step.value())
        print('Restore complete...')
        #sess.run(assign_global_zero_op)
    else:
        print('New model...')
        sess.run(tf.global_variables_initializer())
        print('Initialization complete...')
    
    print('Training...')
    while tf.train.global_step(sess, global_step) * batch_size < length * num_epoch:
        batch = tf.train.global_step(sess, global_step)
        epoch = batch * batch_size / length
        
        x = images[images_completed: images_completed+batch_size]
        y = targets[images_completed: images_completed+batch_size]
        
        images_completed += batch_size
        
        if images_completed >= length:
            images_completed = 0
            epochs_completed += 1

        current_learning_rate = g_learning_rate * learning_rate_decay ** (epoch // learning_rate_decay_step)
        feed_dict = {input: x, ground_truth: y, learning_rate: current_learning_rate}
        
        print('batch no.: [{}], images completed: [{}/{}]'.format(batch, images_completed+batch_size, length))
        
        if batch * batch_size % length == 0:
            print('Processing epoch #{}, Total epoch time: [{}]'.format(epoch + 1, time.time()-epoch_time))
            epoch_time = time.time()
            sess.run(increment_global_step_op)
            predictions, psnr = predict(sess, output, test_images, targets=test_targets)
            feed_dict[validation_score] = np.mean(psnr)
            print('Current PSNR: [{}], Average PSNR: [{}]'.format(psnr, feed_dict[validation_score]))
            _, summary = sess.run([train_step, summary_step], feed_dict=feed_dict)
            saver.save(sess, model_path)
            summary_writer.add_summary(summary, epoch)
        else:
            print('Training epoch #{}, Time: [{}]'.format(epoch + 1, time.time()-start_time))
            sess.run([train_step], feed_dict=feed_dict)
            #predictions, psnr = predict(sess, output, test_images, targets=test_targets)
            #feed_dict[validation_score] = np.mean(psnr)
            #print('Current PSNR: [{}], Average PSNR: [{}]'.format(psnr, feed_dict[validation_score]))
        if batch % 10 == 0:
            saver.save(sess, model_path)
    saver.save(sess, model_path)
num_epoch += 1

In [0]:
num_epoch

In [0]:
!ls data

In [0]:
!zip -r model_VDSR_ckpt31.zip data/model data/log

In [0]:
files.download('model_VDSR_ckpt31.zip')

In [0]:
!ls tmp/

In [0]:
!tensorboard --logdir=data/log

In [0]:
!zip -r log.zip log

In [0]:
files.download('log.zip')

In [0]:
!rm -rf log.zip

In [0]:
!mkdir tmp
!mkdir tmp/log
!cp -r data/log tmp/log

In [0]:
!zip -r output_vdsr.zip output_vdsr

In [0]:
files.download('output_vdsr.zip')

# Tensorboard Trial Code

In [0]:
LOG_DIR = 'data/log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
! curl http://localhost:6006

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1
! unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Trying LocalTunnel

In [0]:
# Install
! npm install -g localtunnel

In [0]:
# Tunnel port 6006 (TensorBoard assumed running)
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1')

In [0]:
# Get url
!cat url.txt

In [0]:
!ls

In [0]:
!cat /usr/local/cuda/version.txt

### Another option

In [0]:
from IPython.display import clear_output, Image, display, HTML
import tensorflow as tf
import numpy as np
from google.colab import files

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [0]:
"""Create a sample tensor"""
#sample_placeholder= tf.placeholder(dtype=tf.float32) 
"""Show it"""
graph_def = tf.get_default_graph().as_graph_def()
show_graph(graph_def)